In [1]:
import pyspark
from pyspark.sql import SparkSession

conf = pyspark.SparkConf().\
        setMaster("spark://master:7077").\
        set("spark.executor.instances", "2") 
spark = SparkSession.builder.config(conf=conf).getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/24 06:49:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
pip install pyspark

In [ ]:
from pyspark.sql.functions import current_date, current_timestamp, \
    date_add, date_sub, datediff, months_between, to_date, lit, to_timestamp


spark.range(3).\
    withColumn("today", current_date()).show()




dateDF = spark.range(3).\
    withColumn("today", current_date()).\
    withColumn("now", current_timestamp())



dateDF.select("today", date_add("today", 100).alias("100일"), date_sub("today", 100).alias("-100일")).show()



dateDF.withColumn("week_ago", date_sub("today", 7)).\
            select(datediff("week_ago", "today")).show()


dateDF.select(to_date(lit('2024-04-01')).alias('start'), 
             to_date(lit('2024-09-27')).alias('end')).\
        select(months_between("end", "start"), datediff("end", "start")).show()


In [ ]:
# 날짜 데이터 파싱 
dateFormat = "yy/MM/dd"
cleanDataDF = spark.range(1).select(to_date(lit('24/06/24'), dateFormat)).show()


In [ ]:
import requests
import pandas as pd 

url = "http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?serviceKey=34lIyWAciAsJlGtIZ4ltpLy2sLZDR%2BBRWvAv8RgoADNEd%2BKCgHe84XiSRUwL8JMMIubzsFW3ddjcNlhZHhvJIQ%3D%3D&pageNo=1&numOfRows=1000&LAWD_CD=11680&DEAL_YMD=202405"


r= requests.get(url)


import xml.etree.ElementTree as ET
root = ET.fromstring(r.text)
items = root.iter(tag='item')
for x in items:
    for y in x:
        print(y.tag, y.text)

apt =pd.DataFrame([{y.tag : y.text for y in x} for x in items])


apt = pd.DataFrame([{y.tag : y.text for y in x} for x in root.iter(tag='item')])
apt_spark_df = spark.createDataFrame(apt)



apt['월'].apply(lambda x : "{:02d}".format(int(x))) + "-" + apt['일']



In [ ]:
import requests
import pandas as pd 

url = "http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?serviceKey=34lIyWAciAsJlGtIZ4ltpLy2sLZDR%2BBRWvAv8RgoADNEd%2BKCgHe84XiSRUwL8JMMIubzsFW3ddjcNlhZHhvJIQ%3D%3D&pageNo=1&numOfRows=1000&LAWD_CD=11680&DEAL_YMD=202405"


r= requests.get(url)


In [ ]:
import xml.etree.ElementTree as ET
root = ET.fromstring(r.text)
items = root.iter(tag='item')
for x in items:
    for y in x:
        print(y.tag, y.text)

apt =pd.DataFrame([{y.tag : y.text for y in x} for x in items])


apt = pd.DataFrame([{y.tag : y.text for y in x} for x in root.iter(tag='item')])
apt_spark_df = spark.createDataFrame(apt)



apt['월'].apply(lambda x : "{:02d}".format(int(x))) + "-" + apt['일']


apt['날짜'] = apt[['월', '일']].apply(lambda x : "2024-" + "{:02d}".format(int(x[0])) + "-" + "{:02d}".format(int(x[1])) ,axis=1)


from pyspark.sql.functions import concat 
apt_spark_df.withColumn("날짜", concat(lit('2024-'), "월", lit("-"), "일")).limit(3).show(truncate=False)


from pyspark.sql.functions import concat, format_string,col
apt_spark_df.withColumn("날짜", concat(lit('2024-'), 
                        format_string("%02d", col("월").cast('int')), lit("-"), 
                        format_string("%02d", col("일").cast('int')))).limit(3).show(truncate=False)

from pyspark.sql.functions import concat, format_string,col
apt_spark_df2 = apt_spark_df.withColumn("날짜", concat(lit('2024-'), 
                        format_string("%02d", col("월").cast('int')), lit("-"), 
                        format_string("%02d", col("일").cast('int'))))




apt_spark_df3 = apt_spark_df2.withColumn("날짜", to_date("날짜", "yyyy-MM-dd"))


apt_spark_df3= apt_spark_df3.withColumn("건축년도", col("건축년도").cast('int'))
